In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
sns.set()
import matplotlib.pyplot as plt
#import missingno as msnum

In [2]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_row", 160)
pd.set_option('max_colwidth', 200)

In [4]:
accepted = pd.read_csv('accepted.csv')
accepted.shape

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(2260701, 151)

In [5]:
accepted.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,246

In [7]:
a = accepted[['purpose', 'fico_range_low', 'fico_range_high', 'loan_amnt', 'annual_inc', 'emp_length']]

In [8]:
a.isnull().sum()

purpose                33
fico_range_low         33
fico_range_high        33
loan_amnt              33
annual_inc             37
emp_length         146940
dtype: int64

In [10]:
a = a.dropna()
a.shape

(2113757, 6)

In [12]:
a.isnull().sum()

purpose            0
fico_range_low     0
fico_range_high    0
loan_amnt          0
annual_inc         0
emp_length         0
dtype: int64

In [13]:
a = a.assign(FICO = (a['fico_range_low'] + a['fico_range_high'])/2)
a = a.rename(columns={"FICO":"Risk_Score"})

In [14]:
a = a.assign(DI = (a['loan_amnt'] / a['annual_inc']))

In [15]:
a['purpose'].value_counts()

debt_consolidation    1198442
credit_card            484003
home_improvement       138479
other                  128403
major_purchase          47321
medical                 24865
small_business          23981
car                     22452
moving                  14349
vacation                14246
house                   13129
wedding                  2323
renewable_energy         1349
educational               415
Name: purpose, dtype: int64

In [16]:
a['purpose'] = a['purpose'].replace({'debt_consolidation':'Debt Consolidation',
                                                   'credit_card':'Credit Card Refinancing',
                                                   'home_improvement':'Home Improvement',
                                                   'other':'Other',
                                                   'major_purchase':'Major Purchase',
                                                   'medical':'Medical Expenses',
                                                   'small_business':'Business Loan',
                                                   'car':'Car Financing',
                                                   'vacation':'Vacation',
                                                   'moving':'Moving And Relocation',
                                                   'house':'Home Purchase',
                                                   'wedding':'Wedding',
                                                   'renewable_energy':'Green Loan',
                                                   'educational':'Student Loan'})

In [17]:
a['purpose'].value_counts(dropna=False)

Debt Consolidation         1198442
Credit Card Refinancing     484003
Home Improvement            138479
Other                       128403
Major Purchase               47321
Medical Expenses             24865
Business Loan                23981
Car Financing                22452
Moving And Relocation        14349
Vacation                     14246
Home Purchase                13129
Wedding                       2323
Green Loan                    1349
Student Loan                   415
Name: purpose, dtype: int64

In [18]:
a = a.rename(columns={'purpose':"Loan Type"})

In [19]:
a = a.assign(Accepted = 1)
a

,Loan Type,fico_range_low,fico_range_high,loan_amnt,annual_inc,emp_length,Risk_Score,DI,Accepted
0,Debt Consolidation,675.0,679.0,3600.0,55000.0,10+ years,677.0,0.065455,1
1,Business Loan,715.0,719.0,24700.0,65000.0,10+ years,717.0,0.380000,1
2,Home Improvement,695.0,699.0,20000.0,63000.0,10+ years,697.0,0.317460,1
3,Debt Consolidation,785.0,789.0,35000.0,110000.0,10+ years,787.0,0.318182,1
4,Major Purchase,695.0,699.0,10400.0,104433.0,3 years,697.0,0.099585,1
...,...,...,...,...,...,...,...,...,...
2260694,Home Improvement,665.0,669.0,24000.0,95000.0,7 years,667.0,0.252632,1
2260695,Debt Consolidation,695.0,699.0,24000.0,108000.0,10+ years,697.0,0.222222,1
2260696,Debt Consolidation,705.0,709.0,40000.0,227000.0,9 years,707.0,0.176211,1
2260697,Debt Consolidation,660.0,664.0,24000.0,110000.0,6 years,662.0,0.218182,1


In [21]:
a = a.rename(columns={"emp_length":"Employment Length"})

In [22]:
a = a[['Risk_Score', 'DI', 'Loan Type', 'Employment Length', 'Accepted']]
a

,Risk_Score,DI,Loan Type,Employment Length,Accepted
0,677.0,0.065455,Debt Consolidation,10+ years,1
1,717.0,0.380000,Business Loan,10+ years,1
2,697.0,0.317460,Home Improvement,10+ years,1
3,787.0,0.318182,Debt Consolidation,10+ years,1
4,697.0,0.099585,Major Purchase,3 years,1
...,...,...,...,...,...
2260694,667.0,0.252632,Home Improvement,7 years,1
2260695,697.0,0.222222,Debt Consolidation,10+ years,1
2260696,707.0,0.176211,Debt Consolidation,9 years,1
2260697,662.0,0.218182,Debt Consolidation,6 years,1


In [23]:
a.dtypes

Risk_Score           float64
DI                   float64
Loan Type             object
Employment Length     object
Accepted               int64
dtype: object

In [24]:
a.isnull().sum()

Risk_Score           0
DI                   0
Loan Type            0
Employment Length    0
Accepted             0
dtype: int64

In [25]:
a.shape

(2113757, 5)

In [26]:
a.to_csv('Accepted_DropNAs.csv', index=False)